In [1]:
import requests
from bs4 import BeautifulSoup
import os

In [7]:
url = "https://www.arconstruction.ie/wp-content/uploads/2020/08/A1-1.jpg"
response = requests.get(url)
response

SSLError: HTTPSConnectionPool(host='www.arconstruction.ie', port=443): Max retries exceeded with url: /wp-content/uploads/2020/08/A1-1.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)')))

In [3]:
# URL of the gallery page
url = "https://www.arconstruction.ie/gallery/"
output_folder = "../gallery"

# Create folder if not exists
os.makedirs(output_folder, exist_ok=True)

# Fetch the page
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to fetch page: {response.status_code}")
    exit()

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Find all image tags
img_tags = soup.find_all("img")

ConnectTimeout: HTTPSConnectionPool(host='www.arconstruction.ie', port=443): Max retries exceeded with url: /gallery/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000217061C5E50>, 'Connection to www.arconstruction.ie timed out. (connect timeout=None)'))

In [ ]:

# Extract and download images
for idx, img in enumerate(img_tags, start=1):
    img_url = img.get("src")
    if img_url and img_url.startswith("/"):
        img_url = "https://www.arconstruction.ie" + img_url  # Make absolute URL
    if img_url and img_url.startswith("http"):
        img_data = requests.get(img_url).content
        file_name = os.path.join(output_folder, f"image_{idx}.jpg")
        with open(file_name, "wb") as f:
            f.write(img_data)
        print(f"Downloaded: {file_name}")

print("✅ All images downloaded successfully!")

In [17]:
import json
from pathlib import Path
from datetime import datetime

# --- Config ---
PROJECT_ROOT = Path.cwd().resolve().parent
print(PROJECT_ROOT)
GALLERY_DIR = PROJECT_ROOT / "media/images"            # where your images live
FIXTURES_DIR = PROJECT_ROOT / "fixtures"
OUTPUT_FIXTURE = FIXTURES_DIR / "sample_gallery.json"


C:\Users\2185060\TCS\Accounts\Visteon\Dev\Repos\arconstruction\backend


In [18]:
# --- Defaults & rules ---
DEFAULT_TAG = "general"  # fallback when no rule matches

# Tags to auto-assign based on filename keywords (adjust as you like)
TAG_RULES = {
    "shop": "shopfitting",
    "fitout": "shopfitting",
    "kitchen": "kitchen",
    "bath": "bathroom",
    "bathroom": "bathroom",
    "tile": "tiling",
    "floor": "flooring",
    "ceiling": "ceiling",
    "plaster": "plastering",
    "paint": "painting",
    "reno": "renovation",
    "refurb": "renovation",
}

ALLOWED_EXT = {".jpg", ".jpeg", ".png", ".webp"}


def guess_tags(filename: str):
    """
    Derive tags from the filename using TAG_RULES (case-insensitive).
    Guarantees at least DEFAULT_TAG if no rule matches.
    """
    name = (filename or "").lower()
    tags = {tag for key, tag in TAG_RULES.items() if key in name}
    return sorted(tags or {DEFAULT_TAG})


def title_from_filename(filename: str):
    """
    Convert filename like 'kitchen_before.jpg' -> 'Kitchen Before'.
    """
    stem = Path(filename).stem
    title = stem.replace("_", " ").replace("-", " ").strip().title()
    return title or stem


def main():
    if not GALLERY_DIR.exists():
        print(f"[!] Gallery folder not found: {GALLERY_DIR}")
        print("    Create it and place your images there, then run this script again.")
        return

    FIXTURES_DIR.mkdir(parents=True, exist_ok=True)
    items = []
    pk = 1

    # Stable timestamp for deterministic fixtures
    ts = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

    files = [p for p in sorted(GALLERY_DIR.iterdir()) if p.is_file() and p.suffix.lower() in ALLOWED_EXT]
    if not files:
        print(f"[!] No images with extensions {sorted(ALLOWED_EXT)} found in: {GALLERY_DIR}")
        return

    for file in files:
        rel_image_path = f"gallery/{file.name}"  # <-- relative to MEDIA_ROOT
        item = {
            "model": "mediahub.galleryitem",
            "pk": pk,
            "fields": {
                "title": title_from_filename(file.name),
                "image": rel_image_path,
                "caption": "",
                "tags": guess_tags(file.name),  # <-- now always at least [DEFAULT_TAG]
                "is_active": True,
                "created_at": ts,
            }
        }
        items.append(item)
        pk += 1

    OUTPUT_FIXTURE.write_text(json.dumps(items, indent=2), encoding="utf-8")
    print(f"[✓] Wrote {len(items)} records to {OUTPUT_FIXTURE}")
    print(f"    Files scanned from: {GALLERY_DIR}")
    print("    Then load fixture:")
    print("    python manage.py loaddata fixtures/sample_gallery.json")


if __name__ == "__main__":
    main()

[✓] Wrote 24 records to C:\Users\2185060\TCS\Accounts\Visteon\Dev\Repos\arconstruction\backend\fixtures\sample_gallery.json
    Files scanned from: C:\Users\2185060\TCS\Accounts\Visteon\Dev\Repos\arconstruction\backend\media\images
    Then load fixture:
    python manage.py loaddata fixtures/sample_gallery.json


C:\Users\2185060\AppData\Local\Temp\ipykernel_14684\3710780028.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
